In [5]:

import logging

import colorlog
from neo4j import GraphDatabase
from pydantic_settings import BaseSettings, SettingsConfigDict


In [7]:
class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file=".env", env_ignore_empty=True, extra="ignore"
    )

    NEO4J_URI: str
    NEO4J_USERNAME: str
    NEO4J_PASSWORD: str
    NEO4J_DATABASE: str
    NEO4J_AURA_INSTANCEID: str
    NEO4J_AURA_INSTANCENAME: str


settings = Settings()

In [8]:
def setup_logger(name: str='8opt') -> logging.Logger:
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    # Remove all existing handlers
    logger.handlers.clear()

    handler = logging.StreamHandler()

    # Define color scheme for different log levels
    formatter = colorlog.ColoredFormatter(
        "%(log_color)s%(asctime)s - %(name)s - %(levelname)s - %(message)s",
        datefmt="%y-%m-%d %H:%M:%S",
        log_colors={
            "DEBUG": "cyan",
            "INFO": "green",
            "WARNING": "yellow",
            "ERROR": "red",
            "CRITICAL": "red,bg_white",
        },
    )

    handler.setFormatter(formatter)
    logger.addHandler(handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger


logger = setup_logger()


In [10]:
with GraphDatabase.driver(
    settings.NEO4J_URI, auth=(settings.NEO4J_USERNAME, settings.NEO4J_PASSWORD)
) as driver:
    driver.verify_connectivity()
    logger.info("Connection established.")

25-09-07 20:13:14 - 8opt - INFO - Connection established.


In [16]:
summary = driver.execute_query("""
    CREATE (a:Person {name: $name})
    CREATE (b:Person {name: $friendName})
    CREATE (a)-[:KNOWS]->(b)
    """,
    name="Alice", friendName="David",
    database_=settings.NEO4J_DATABASE,
).summary


logger.info("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

/tmp/ipykernel_36314/1836581358.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  summary = driver.execute_query("""
25-09-07 20:18:19 - 8opt - INFO - Created 2 nodes in 24 ms.


In [21]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person)-[:KNOWS]->(:Person)
    RETURN p.name AS name
    """,
    database_=settings.NEO4J_DATABASE,
)

# Loop through results and do something with them
for record in records:
    logger.debug(record.data())  # obtain record as dict

# Summary information
logger.info("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

/tmp/ipykernel_36314/2601654325.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""
25-09-07 20:30:20 - 8opt - DEBUG - {'name': 'Alice'}
25-09-07 20:30:20 - 8opt - DEBUG - {'name': 'Alice'}
25-09-07 20:30:20 - 8opt - INFO - The query `
    MATCH (p:Person)-[:KNOWS]->(:Person)
    RETURN p.name AS name
    ` returned 2 records in 6 ms.


In [22]:
# To find David's node
result = driver.execute_query(
    "MATCH (p:Person {name: $name}) RETURN p.name AS name",
    name="David",
    database_=settings.NEO4J_DATABASE
)

# The result will contain records with the name property
for record in result.records:
    logger.info(f"Found person: {record['name']}")

/tmp/ipykernel_36314/1649563956.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  result = driver.execute_query(
25-09-07 20:32:37 - 8opt - INFO - Found person: David
25-09-07 20:32:37 - 8opt - INFO - Found person: David


In [23]:
# To find who Alice knows
result = driver.execute_query("""
    MATCH (a:Person {name: $name})-[:KNOWS]->(friend)
    RETURN friend.name AS friend_name
    """,
    name="Alice",
    database_=settings.NEO4J_DATABASE
)

for record in result.records:
    logger.info(f"Alice knows: {record['friend_name']}")

/tmp/ipykernel_36314/3087214056.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  result = driver.execute_query("""
25-09-07 20:32:54 - 8opt - INFO - Alice knows: David
25-09-07 20:32:54 - 8opt - INFO - Alice knows: David


In [24]:
# Get the database schema
result = driver.execute_query("""
    CALL db.schema.visualization()
    YIELD nodes, relationships
    RETURN nodes, relationships
    """,
    database_=settings.NEO4J_DATABASE
)

# Print the schema information
for record in result.records:
    logger.info("Nodes (labels and properties):")
    for node in record["nodes"]:
        logger.info(f"  - {node}")
    
    logger.info("\nRelationships:")
    for rel in record["relationships"]:
        logger.info(f"  - {rel}")

/tmp/ipykernel_36314/3219428506.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  result = driver.execute_query("""
25-09-07 20:33:46 - 8opt - INFO - Nodes (labels and properties):
25-09-07 20:33:46 - 8opt - INFO -   - <Node element_id='-103' labels=frozenset({'Person'}) properties={'name': 'Person', 'indexes': [], 'constraints': []}>
25-09-07 20:33:46 - 8opt - INFO - 
Relationships:
25-09-07 20:33:46 - 8opt - INFO -   - <Relationship element_id='-103' nodes=(<Node element_id='-103' labels=frozenset({'Person'}) properties={'name': 'Person', 'indexes': [], 'constraints': []}>, <Node element_id='-103' labels=frozenset({'Person'}) properties={'name': 'Person', 'indexes': [], 'constraints': []}>) type='KNOWS' properties={'name': 'KNOWS'}>


In [ ]:
# First, let's create some people and their jobs
result = driver.execute_query("""
    // Create people
    CREATE (alice:Person {name: 'Alice', age: 30})
    CREATE (bob:Person {name: 'Bob', age: 35})
    CREATE (charlie:Person {name: 'Charlie', age: 28})
    CREATE (david:Person {name: 'David', age: 40})
    
    // Create jobs
    CREATE (engineer:Job {title: 'Software Engineer', department: 'Engineering'})
    CREATE (manager:Job {title: 'Project Manager', department: 'Management'})
    CREATE (designer:Job {title: 'UI/UX Designer', department: 'Design'})
    CREATE (analyst:Job {title: 'Data Analyst', department: 'Analytics'})
    
    // Create relationships (people to jobs)
    CREATE (alice)-[r1:WORKS_AS]->(engineer)
    CREATE (bob)-[r2:WORKS_AS]->(manager)
    CREATE (charlie)-[r3:WORKS_AS]->(designer)
    CREATE (david)-[r4:WORKS_AS]->(analyst)
    CREATE (alice)-[r5:WORKS_AS]->(analyst)  // Alice has two jobs
    
    // Create relationships between people
    CREATE (alice)-[:KNOWS]->(bob)
    CREATE (alice)-[:KNOWS]->(charlie)
    CREATE (bob)-[:MANAGES]->(charlie)
    CREATE (bob)-[:MANAGES]->(david)
    CREATE (charlie)-[:COLLABORATES_WITH]->(david)
    
    // Add some properties to relationships
    SET r1.since = 2019
    SET r2.since = 2018
    SET r3.since = 2020
    SET r4.since = 2017
    SET r5.since = 2021, r5.is_primary = false
""", database_=settings.NEO4J_DATABASE)

logger.info("Created graph with people, jobs, and relationships")

In [18]:
# session.close()
driver.close()